In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd


In [2]:
#Load, filter and concatenate individual variant libraries

variant_library_df = pd.read_csv("apa_100_variants_rev2_20220621_pred.csv", sep='\t')
variant_library_df['ref_master_seq'] = variant_library_df['ref_seq'].str.slice(20, 20+164)
variant_library_df['var_master_seq'] = variant_library_df['var_seq'].str.slice(20, 20+164)


In [3]:

variant_library_df


,gene,gene_id,chrom,strand,snp_id,data_source,experiment,pas_pos,pas_pos_hg38,var_position,...,ref_iso_corrected_77_127,var_iso_corrected_77_127,delta_iso_corrected_77_127,ref_logit_corrected_0_205,var_logit_corrected_0_205,ref_iso_corrected_0_205,var_iso_corrected_0_205,delta_iso_corrected_0_205,ref_master_seq,var_master_seq
0,CYP4F22,CYP4F22.1,Missing,Missing,Missing,Array_2019,control_CYP4F22.1,-1,-1,-1,...,0.020933,0.020933,0.000000,-3.842118,-3.842118,0.020998,0.020998,0.000000,AACTGAGCGAAGCTGCGGGCTAGGAGGCTGGAAAGCCATTTATCCT...,AACTGAGCGAAGCTGCGGGCTAGGAGGCTGGAAAGCCATTTATCCT...
1,BBS2,BBS2.3,Missing,Missing,Missing,Array_2019,control_BBS2.3,-1,-1,-1,...,0.082249,0.082249,0.000000,-0.275216,-0.275216,0.431627,0.431627,0.000000,TGCTTTTGAAGTATTAGGGCTTCAGAGTCCAATTTTGCTTATTTAT...,TGCTTTTGAAGTATTAGGGCTTCAGAGTCCAATTTTGCTTATTTAT...
2,CANT1,CANT1.3,Missing,Missing,Missing,Array_2019,control_CANT1.3,-1,-1,-1,...,0.221213,0.221213,0.000000,-1.131754,-1.131754,0.243838,0.243838,0.000000,CAGCATAATGATTGTGTTCCTTCCCTATGGGATCCATCTGTTTTGT...,CAGCATAATGATTGTGTTCCTTCCCTATGGGATCCATCTGTTTTGT...
3,COL4A4,COL4A4.1,Missing,Missing,Missing,Array_2019,control_COL4A4.1,-1,-1,-1,...,0.412868,0.412868,0.000000,-0.163351,-0.163351,0.459253,0.459253,0.000000,TTGGCACTTAAATATTTGTTGTATTACGATTTATAATATGGTTCAT...,TTGGCACTTAAATATTTGTTGTATTACGATTTATAATATGGTTCAT...
4,RYR2,RYR2.1,Missing,Missing,Missing,Array_2019,control_RYR2.1,-1,-1,-1,...,0.543832,0.543832,0.000000,0.477521,0.477521,0.617162,0.617162,0.000000,GGGAAACAAATTAAAGTGATAATCCATTGTAGTTCTGCAAATGAAA...,GGGAAACAAATTAAAGTGATAATCCATTGTAGTTCTGCAAATGAAA...
5,RB1,RB1.11,Missing,Missing,Missing,Array_2019,control_RB1.11,-1,-1,-1,...,0.814467,0.814467,0.000000,1.884298,1.884298,0.868104,0.868104,0.000000,CGCTCTTGAGGTTGTAATGGCCACATATAGCAGTAAGTTAAATTTT...,CGCTCTTGAGGTTGTAATGGCCACATATAGCAGTAAGTTAAATTTT...
6,CRLS1,CRLS1.9,chr20,+,rs2235816,aQTL,brain_specific_variant,6018668,-1,6018711,...,0.205982,0.318474,0.112492,-1.179113,-0.560694,0.235212,0.363387,0.128175,AAAGGTTTTTTAAACTTTGGAGACTCTTTCTTTTGTTAAGCAGTTA...,AAAGGTTTTTTAAACTTTGGAGACTCTTTCTTTTGTTAAGCAGTTA...
7,ALDH7A1,ALDH7A1.17,chr5,-,rs744722,aQTL,brain_specific_variant,125878941,-1,125878908,...,0.471706,0.370828,-0.100878,0.133151,-0.312776,0.533239,0.422437,-0.110801,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...
8,ALDH7A1,ALDH7A1.17,chr5,-,rs744722_control,aQTL,brain_specific_variant,125878941,-1,125878908,...,0.484108,0.490019,0.005911,0.131926,0.157851,0.532934,0.539381,0.006447,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...,TTCTACATGTTTCTAGCACAAGGACTCCATTTTTTTTGGTTATAAT...
9,F2,F2.1,chr11,+,rs1799963,gnomAD_F2,variant,46761035,46739485,46761055,...,0.201017,0.256406,0.055389,-1.253825,-0.924560,0.222039,0.284030,0.061991,AACCAATCCCGTGAAAGAATTATTTTTGTGTTTCTAAAACTATGGT...,AACCAATCCCGTGAAAGAATTATTTTTGTGTTTCTAAAACTATGGT...


In [4]:
#Load oligo array from 2019

import isolearn.io as isoio

isoform_pseudo_count = 1.
proximal_start = 0
proximal_end = 205

file_prefix = str(proximal_start) + "_" + str(proximal_end)

seq_dict = isoio.load('../../../../aparent/data/prepared_data/apa_array_data/apa_array_data_seq')

#Load predictions (Resnet)
model_name = 'aparent_all_libs_resnet_no_clinvar_wt_ep_5'
pred_dict = isoio.load('../../../analysis/predictions/apa_array_data_legacy/' + model_name + '_predictions_seq')


In [5]:

print("n (sequences) = " + str(len(seq_dict['array_df'])))


n (sequences) = 186066


In [6]:

seq_df = seq_dict['array_df']
seq_cuts = seq_dict['pooled_cuts']

pred_df = pred_dict['array_df']
cuts_pred = pred_dict['cut_prob']

seq_df['row_index_true'] = np.arange(len(seq_df), dtype=np.int)
pred_df['row_index_pred'] = np.arange(len(pred_df), dtype=np.int)

seq_df = seq_df.join(pred_df[['seq', 'row_index_pred']].set_index('seq'), on='seq', how='inner').copy().reset_index(drop=True)

seq_cuts = seq_cuts[np.ravel(seq_df['row_index_true'].values), :]
cut_true = np.concatenate([np.array(seq_cuts[:, 180: 180 + 205].todense()), np.array(seq_cuts[:, -1].todense()).reshape(-1, 1)], axis=-1)# - 1

seq_df['proximal_count'] = [np.sum(cut_true[i, proximal_start:proximal_end]) for i in range(len(seq_df))]
seq_df['total_count'] = [np.sum(cut_true[i, :]) for i in range(len(seq_df))]

seq_df['iso_true'] = (seq_df['proximal_count'] + isoform_pseudo_count) / (seq_df['total_count'] + 2. * isoform_pseudo_count)
seq_df['logodds_true'] = np.log(seq_df['iso_true'] / (1.0 - seq_df['iso_true']))

cut_pred = np.array(cuts_pred[np.ravel(seq_df['row_index_pred'].values), :].todense())

seq_df['iso_pred'] = [np.clip(np.sum(cut_pred[i, proximal_start:proximal_end]), 1e-6, 1. - 1e-6) for i in range(len(seq_df))]
seq_df['logodds_pred'] = np.log(seq_df['iso_pred'] / (1.0 - seq_df['iso_pred']))


In [7]:
#ALDH7A1 variants

seq_df.query("gene == 'ALDH7A1.2' and wt_seq != 'Unmapped'")['clinvar_id'].unique()


array(['NM_001182.4(ALDH7A1):c.*1741T>A', 'Missing',
       'NM_001182.4(ALDH7A1):c.*1749G>T',
       'NM_001182.4(ALDH7A1):c.*1730C>T',
       'NM_001182.4(ALDH7A1):c.*1748T>A',
       'NM_001182.4(ALDH7A1):c.*1804C>T',
       'NM_001182.4(ALDH7A1):c.*1693A>C'], dtype=object)

In [8]:
#Filter on high quality

print("n (pre-filtering) = " + str(len(seq_df)))

seq_df = seq_df.query("n_barcodes >= 1 and mean_total_count >= 200").copy().reset_index(drop=True)
seq_df = seq_df.loc[seq_df['experiment'].isin([
    'sensitive_genes',
    'acmg_apadb',
    'human variant',
    'intronic_pas',
    'clinvar_mut',
    'acmg_polyadb',
    'clinvar_wt'
])].copy().reset_index(drop=True)

seq_df = seq_df.loc[~seq_df['seq'].str.slice(0, 20).str.contains("AAAAAAAAA|AAAAGAAAA|AAAACAAAA|AAAATAAAA")].copy().reset_index(drop=True)

seq_df = seq_df.drop_duplicates(subset=['master_seq'], keep='first')
seq_df = seq_df.sort_values(by=['gene'], ascending=True).copy().reset_index(drop=True)

print("n (post-filtering) = " + str(len(seq_df)))


n (pre-filtering) = 186066
n (post-filtering) = 28552


In [9]:

variant_library_df.join(seq_df[['master_seq', 'logodds_true', 'iso_true']].set_index('master_seq'), on='ref_master_seq', how='inner')


,gene,gene_id,chrom,strand,snp_id,data_source,experiment,pas_pos,pas_pos_hg38,var_position,...,delta_iso_corrected_77_127,ref_logit_corrected_0_205,var_logit_corrected_0_205,ref_iso_corrected_0_205,var_iso_corrected_0_205,delta_iso_corrected_0_205,ref_master_seq,var_master_seq,logodds_true,iso_true
0,CYP4F22,CYP4F22.1,Missing,Missing,Missing,Array_2019,control_CYP4F22.1,-1,-1,-1,...,0.000000,-3.842118,-3.842118,0.020998,0.020998,0.000000,AACTGAGCGAAGCTGCGGGCTAGGAGGCTGGAAAGCCATTTATCCT...,AACTGAGCGAAGCTGCGGGCTAGGAGGCTGGAAAGCCATTTATCCT...,-4.239665,0.014208
1,BBS2,BBS2.3,Missing,Missing,Missing,Array_2019,control_BBS2.3,-1,-1,-1,...,0.000000,-0.275216,-0.275216,0.431627,0.431627,0.000000,TGCTTTTGAAGTATTAGGGCTTCAGAGTCCAATTTTGCTTATTTAT...,TGCTTTTGAAGTATTAGGGCTTCAGAGTCCAATTTTGCTTATTTAT...,-0.497273,0.378182
2,CANT1,CANT1.3,Missing,Missing,Missing,Array_2019,control_CANT1.3,-1,-1,-1,...,0.000000,-1.131754,-1.131754,0.243838,0.243838,0.000000,CAGCATAATGATTGTGTTCCTTCCCTATGGGATCCATCTGTTTTGT...,CAGCATAATGATTGTGTTCCTTCCCTATGGGATCCATCTGTTTTGT...,-1.383259,0.200486
3,COL4A4,COL4A4.1,Missing,Missing,Missing,Array_2019,control_COL4A4.1,-1,-1,-1,...,0.000000,-0.163351,-0.163351,0.459253,0.459253,0.000000,TTGGCACTTAAATATTTGTTGTATTACGATTTATAATATGGTTCAT...,TTGGCACTTAAATATTTGTTGTATTACGATTTATAATATGGTTCAT...,0.106060,0.526490
4,RYR2,RYR2.1,Missing,Missing,Missing,Array_2019,control_RYR2.1,-1,-1,-1,...,0.000000,0.477521,0.477521,0.617162,0.617162,0.000000,GGGAAACAAATTAAAGTGATAATCCATTGTAGTTCTGCAAATGAAA...,GGGAAACAAATTAAAGTGATAATCCATTGTAGTTCTGCAAATGAAA...,2.995732,0.952381
5,RB1,RB1.11,Missing,Missing,Missing,Array_2019,control_RB1.11,-1,-1,-1,...,0.000000,1.884298,1.884298,0.868104,0.868104,0.000000,CGCTCTTGAGGTTGTAATGGCCACATATAGCAGTAAGTTAAATTTT...,CGCTCTTGAGGTTGTAATGGCCACATATAGCAGTAAGTTAAATTTT...,3.538057,0.971751
7,ALDH7A1,ALDH7A1.17,chr5,-,rs744722,aQTL,brain_specific_variant,125878941,-1,125878908,...,-0.100878,0.133151,-0.312776,0.533239,0.422437,-0.110801,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...,-1.231101,0.225989
8,ALDH7A1,ALDH7A1.17,chr5,-,rs744722_control,aQTL,brain_specific_variant,125878941,-1,125878908,...,0.005911,0.131926,0.157851,0.532934,0.539381,0.006447,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...,TTCTACATGTTTCTAGCACAAGGACTCCATTTTTTTTGGTTATAAT...,-1.231101,0.225989
9,F2,F2.1,chr11,+,rs1799963,gnomAD_F2,variant,46761035,46739485,46761055,...,0.055389,-1.253825,-0.924560,0.222039,0.284030,0.061991,AACCAATCCCGTGAAAGAATTATTTTTGTGTTTCTAAAACTATGGT...,AACCAATCCCGTGAAAGAATTATTTTTGTGTTTCTAAAACTATGGT...,-2.354017,0.086747
11,F2,F2.1,chr11,+,rs1799963_control,gnomAD_F2,gnomad_control,46761035,46739485,46761054,...,0.005252,-1.185032,-1.152269,0.234149,0.240075,0.005926,AACCAATCCCGTGAAAGAATTATTTTTGTGTTTCTAAAACTATGGT...,AACCAATCCCGTGAAAGAATTATTTTTGTGTTTCTAAAACTATGGT...,-2.354017,0.086747


In [10]:

variant_library_df.join(seq_df[['master_seq', 'logodds_true', 'iso_true']].set_index('master_seq'), on='var_master_seq', how='inner')


,gene,gene_id,chrom,strand,snp_id,data_source,experiment,pas_pos,pas_pos_hg38,var_position,...,delta_iso_corrected_77_127,ref_logit_corrected_0_205,var_logit_corrected_0_205,ref_iso_corrected_0_205,var_iso_corrected_0_205,delta_iso_corrected_0_205,ref_master_seq,var_master_seq,logodds_true,iso_true
0,CYP4F22,CYP4F22.1,Missing,Missing,Missing,Array_2019,control_CYP4F22.1,-1,-1,-1,...,0.000000,-3.842118,-3.842118,0.020998,0.020998,0.000000,AACTGAGCGAAGCTGCGGGCTAGGAGGCTGGAAAGCCATTTATCCT...,AACTGAGCGAAGCTGCGGGCTAGGAGGCTGGAAAGCCATTTATCCT...,-4.239665,0.014208
1,BBS2,BBS2.3,Missing,Missing,Missing,Array_2019,control_BBS2.3,-1,-1,-1,...,0.000000,-0.275216,-0.275216,0.431627,0.431627,0.000000,TGCTTTTGAAGTATTAGGGCTTCAGAGTCCAATTTTGCTTATTTAT...,TGCTTTTGAAGTATTAGGGCTTCAGAGTCCAATTTTGCTTATTTAT...,-0.497273,0.378182
2,CANT1,CANT1.3,Missing,Missing,Missing,Array_2019,control_CANT1.3,-1,-1,-1,...,0.000000,-1.131754,-1.131754,0.243838,0.243838,0.000000,CAGCATAATGATTGTGTTCCTTCCCTATGGGATCCATCTGTTTTGT...,CAGCATAATGATTGTGTTCCTTCCCTATGGGATCCATCTGTTTTGT...,-1.383259,0.200486
3,COL4A4,COL4A4.1,Missing,Missing,Missing,Array_2019,control_COL4A4.1,-1,-1,-1,...,0.000000,-0.163351,-0.163351,0.459253,0.459253,0.000000,TTGGCACTTAAATATTTGTTGTATTACGATTTATAATATGGTTCAT...,TTGGCACTTAAATATTTGTTGTATTACGATTTATAATATGGTTCAT...,0.106060,0.526490
4,RYR2,RYR2.1,Missing,Missing,Missing,Array_2019,control_RYR2.1,-1,-1,-1,...,0.000000,0.477521,0.477521,0.617162,0.617162,0.000000,GGGAAACAAATTAAAGTGATAATCCATTGTAGTTCTGCAAATGAAA...,GGGAAACAAATTAAAGTGATAATCCATTGTAGTTCTGCAAATGAAA...,2.995732,0.952381
5,RB1,RB1.11,Missing,Missing,Missing,Array_2019,control_RB1.11,-1,-1,-1,...,0.000000,1.884298,1.884298,0.868104,0.868104,0.000000,CGCTCTTGAGGTTGTAATGGCCACATATAGCAGTAAGTTAAATTTT...,CGCTCTTGAGGTTGTAATGGCCACATATAGCAGTAAGTTAAATTTT...,3.538057,0.971751
7,ALDH7A1,ALDH7A1.17,chr5,-,rs744722,aQTL,brain_specific_variant,125878941,-1,125878908,...,-0.100878,0.133151,-0.312776,0.533239,0.422437,-0.110801,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...,-1.394077,0.198758
8,ALDH7A1,ALDH7A1.17,chr5,-,rs744722_control,aQTL,brain_specific_variant,125878941,-1,125878908,...,0.005911,0.131926,0.157851,0.532934,0.539381,0.006447,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...,TTCTACATGTTTCTAGCACAAGGACTCCATTTTTTTTGGTTATAAT...,-1.075754,0.254310
9,F2,F2.1,chr11,+,rs1799963,gnomAD_F2,variant,46761035,46739485,46761055,...,0.055389,-1.253825,-0.924560,0.222039,0.284030,0.061991,AACCAATCCCGTGAAAGAATTATTTTTGTGTTTCTAAAACTATGGT...,AACCAATCCCGTGAAAGAATTATTTTTGTGTTTCTAAAACTATGGT...,-2.678450,0.064257
11,F2,F2.1,chr11,+,rs1799963_control,gnomAD_F2,gnomad_control,46761035,46739485,46761054,...,0.005252,-1.185032,-1.152269,0.234149,0.240075,0.005926,AACCAATCCCGTGAAAGAATTATTTTTGTGTTTCTAAAACTATGGT...,AACCAATCCCGTGAAAGAATTATTTTTGTGTTTCTAAAACTATGGT...,-1.952690,0.124260


In [13]:

print(variant_library_df.query("gene_id == 'ALDH7A1.17'").iloc[0]['ref_seq_w_homology'])
print(variant_library_df.query("gene_id == 'ALDH7A1.17'").iloc[0]['var_seq_w_homology'])
print("")
print(variant_library_df.query("gene_id == 'ALDH7A1.17'").iloc[0]['ref_master_seq'])
print(variant_library_df.query("gene_id == 'ALDH7A1.17'").iloc[0]['var_master_seq'])


TCCCTACACGACGCTCTTCCGATCTAAACAAATTTATAATTTTTCTTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAATGTTCATTAAAACATATGCCTTCAATTTGTTCCTCTTTGTTCTTTTTTGCTGATTTAGCAGATATAAATCACTTGTTAGTATTTTTAACTATCCAAGTTAAAATGTACATTTGTAAGTGGGAGCAGATACTGGCTTAACTATGC
TCCCTACACGACGCTCTTCCGATCTAAACAAATTTATAATTTTTCTTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAATGTTCATTAAAACATATGCCTTCAATTTGTTCCTCTTTTTTCTTTTTTGCTGATTTAGCAGATATAAATCACTTGTTAGTATTTTTAACTATCCAAGTTAAAATGTACATTTGTAAGTGGGAGCAGATACTGGCTTAACTATGC

TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAATGTTCATTAAAACATATGCCTTCAATTTGTTCCTCTTTGTTCTTTTTTGCTGATTTAGCAGATATAAATCACTTGTTAGTATTTTTAACTATCCAAGTTAAAATGTACATTTGTAAGTG
TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAATGTTCATTAAAACATATGCCTTCAATTTGTTCCTCTTTTTTCTTTTTTGCTGATTTAGCAGATATAAATCACTTGTTAGTATTTTTAACTATCCAAGTTAAAATGTACATTTGTAAGTG


In [14]:

print(variant_library_df.query("gene_id == 'ALDH7A1.17'").iloc[1]['ref_seq_w_homology'])
print(variant_library_df.query("gene_id == 'ALDH7A1.17'").iloc[1]['var_seq_w_homology'])
print("")
print(variant_library_df.query("gene_id == 'ALDH7A1.17'").iloc[1]['ref_master_seq'])
print(variant_library_df.query("gene_id == 'ALDH7A1.17'").iloc[1]['var_master_seq'])


TCCCTACACGACGCTCTTCCGATCTCATCTACTTACCAGTTCCTGTTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAATGTTCATTAAAACATATGCCTTCAATTTGTTCCTCTTTGTTCTTTTTTGCTGATTTAGCAGATATAAATCACTTGTTAGTATTTTTAACTATCCAAGTTAAAATGTACATTTGTAAGTGGGAGCAGATACTGGCTTAACTATGC
TCCCTACACGACGCTCTTCCGATCTCATCTACTTACCAGTTCCTGTTCTACATGTTTCTAGCACAAGGACTCCATTTTTTTTGGTTATAATGTTCATTAAAACATATGCCTTCAATTTGTTCCTCTTTGTTCTTTTTTGCTGATTTAGCAGATATAAATCACTTGTTAGTATTTTTAACTATCCAAGTTAAAATGTACATTTGTAAGTGGGAGCAGATACTGGCTTAACTATGC

TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAATGTTCATTAAAACATATGCCTTCAATTTGTTCCTCTTTGTTCTTTTTTGCTGATTTAGCAGATATAAATCACTTGTTAGTATTTTTAACTATCCAAGTTAAAATGTACATTTGTAAGTG
TTCTACATGTTTCTAGCACAAGGACTCCATTTTTTTTGGTTATAATGTTCATTAAAACATATGCCTTCAATTTGTTCCTCTTTGTTCTTTTTTGCTGATTTAGCAGATATAAATCACTTGTTAGTATTTTTAACTATCCAAGTTAAAATGTACATTTGTAAGTG


In [26]:

seq_df.query("master_seq == 'TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAATGTTCATTAAAACATATGCCTTCAATTTGTTCCTCTTTGTTCTTTTTTGCTGATTTAGCAGATATAAATCACTTGTTAGTATTTTTAACTATCCAAGTTAAAATGTACATTTGTAAGTG'")


,seq,experiment,subexperiment,gene,significance,clinvar_id,variant,in_acmg,sitetype,wt_seq,...,tgta_pos_3,seq_ext,row_index_true,row_index_pred,proximal_count,total_count,iso_true,logodds_true,iso_pred,logodds_pred
591,ACAGTCCTTTTGAGTACGTTTTCTACATGTTTCTAGCACAAGGACT...,clinvar_wt,ALDH7A1.2,ALDH7A1.2,Missing,Missing,wt,No,UTR3,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...,...,0,TACAAGGCCAAGAAGCCCGTGCAGCTGCCCGGCGCCTACAACGTCA...,13081,13081,79.0,352.0,0.225989,-1.231101,0.553698,0.215622


In [27]:

seq_df.query("master_seq == 'TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAATGTTCATTAAAACATATGCCTTCAATTTGTTCCTCTTTTTTCTTTTTTGCTGATTTAGCAGATATAAATCACTTGTTAGTATTTTTAACTATCCAAGTTAAAATGTACATTTGTAAGTG'")


,seq,experiment,subexperiment,gene,significance,clinvar_id,variant,in_acmg,sitetype,wt_seq,...,tgta_pos_3,seq_ext,row_index_true,row_index_pred,proximal_count,total_count,iso_true,logodds_true,iso_pred,logodds_pred
586,AGAGGTTGGATGAGCGGGAATTCTACATGTTTCTAGCACAAGGACT...,human variant,ALDH7A1.2,ALDH7A1.2,Benign,NM_001182.4(ALDH7A1):c.*1749G>T,snv,No,UTR3,TTCTACATGTTTCTAGCACAAGGACTCAATTTTTTTTGGTTATAAT...,...,0,TACAAGGCCAAGAAGCCCGTGCAGCTGCCCGGCGCCTACAACGTCA...,24870,24870,95.0,481.0,0.198758,-1.394077,0.405042,-0.384501
